This notebook uses pretrained model -  
*   [TFDistilBertForSequenceClassification](https://huggingface.co/transformers/model_doc/distilbert.html#tfdistilbertforsequenceclassification)

and the dataset - 
*   [Laurence Moroney - Sarcasm dataset](https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json)



In [1]:
!pip install transformers

     |████████████████████████████████| 1.8MB 22.1MB/s 
     |████████████████████████████████| 2.9MB 54.7MB/s 
     |████████████████████████████████| 890kB 52.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=46a47928d188d60b57afe148baf4c752a279b4b573e236defc4d7ecea88010d6
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification

In [3]:
import tensorflow as tf

In [10]:
import json

Data preparation

In [4]:
cd /content/drive/MyDrive/HuggingFace_Pretrained_FineTuning/Laurence_Moroney

/content/drive/MyDrive/HuggingFace_Pretrained_FineTuning/Laurence_Moroney


In [6]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json \
    -O /content/drive/MyDrive/HuggingFace_Pretrained_FineTuning/Laurence_Moroney/sarcasm.json

--2021-01-16 05:21:37--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.9.208, 142.250.31.128, 172.217.15.80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.9.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘/content/drive/MyDrive/HuggingFace_Pretrained_FineTuning/Laurence_Moroney/sarcasm.json’

/content/drive/MyDr 100%[===================>]   5.38M  --.-KB/s    in 0.1s    

2021-01-16 05:21:38 (47.3 MB/s) - ‘/content/drive/MyDrive/HuggingFace_Pretrained_FineTuning/Laurence_Moroney/sarcasm.json’ saved [5643545/5643545]



In [7]:
training_size = 20000

In [8]:
dataFilePath = '/content/drive/MyDrive/HuggingFace_Pretrained_FineTuning/Laurence_Moroney/sarcasm.json'

In [11]:
with open(dataFilePath, 'r') as f:
    datastore = json.load(f)

In [14]:
sentences = []
labels = []
urls = []
for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic']) 

In [15]:
print('Example Sentence : ', sentences[0])
print('Example Label : ', labels[0])

Example Sentence :  former versace store clerk sues over secret 'black code' for minority shoppers
Example Label :  0


In [16]:
training_sentences = sentences[0:training_size]
validation_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
validation_labels = labels[training_size:]

In [18]:
print('Total Training Data : ', len(training_sentences))
print('Total Validation Data : ', len(validation_labels))

Total Training Data :  20000
Total Validation Data :  6709


BERT Set up and Training

In [19]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

Tokenization

In [20]:
train_encodings = tokenizer(training_sentences,
                            truncation=True,
                            padding=True)
val_encodings = tokenizer(validation_sentences,
                            truncation=True,
                            padding=True)

In [22]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    training_labels
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    validation_labels
))

Fetch Pre-Trained model from HuggingFace. In this case we're using [distilbert-base-uncased](https://huggingface.co/distilbert-base-uncased)

In [27]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
#Here, num_labels represents the number of labels in the dataset.

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'vocab_transform', 'vocab_projector', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'dropout_19', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [28]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])
model.fit(train_dataset.shuffle(100).batch(16),
          epochs=3,
          batch_size=16,
          validation_data=val_dataset.shuffle(100).batch(16))

Epoch 1/3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7f69f7ccfd90> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7f69f7ccfd90> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


1250/1250 [==============================] - ETA: 0s - loss: 0.3397 - accuracy: 0.8436

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


1250/1250 [==============================] - 201s 136ms/step - loss: 0.3396 - accuracy: 0.8437 - val_loss: 0.2084 - val_accuracy: 0.9128
Epoch 2/3
1250/1250 [==============================] - 170s 136ms/step - loss: 0.1160 - accuracy: 0.9574 - val_loss: 0.2712 - val_accuracy: 0.9115
Epoch 3/3
1250/1250 [==============================] - 170s 136ms/step - loss: 0.0460 - accuracy: 0.9856 - val_loss: 0.3175 - val_accuracy: 0.9197


In [29]:
model.save_pretrained("/content/drive/MyDrive/HuggingFace_Pretrained_FineTuning/Laurence_Moroney/sentiment_custom_model")

Loading the saved model for prediction

In [31]:
modelPath = "/content/drive/MyDrive/HuggingFace_Pretrained_FineTuning/Laurence_Moroney/sentiment_custom_model"

In [32]:
loaded_model = TFDistilBertForSequenceClassification.from_pretrained(modelPath)

Some layers from the model checkpoint at /content/drive/MyDrive/HuggingFace_Pretrained_FineTuning/Laurence_Moroney/sentiment_custom_model were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/HuggingFace_Pretrained_FineTuning/Laurence_Moroney/sentiment_custom_model and are newly initialized: ['dropout_59']
You should probably TRAIN

Not Sarcasm prediction

In [40]:
test_sentence_not_sarcasm = "With their homes in ashes, residents share harrowing tales of survival after massive wildfires kill 15"

In [34]:
predict_input_not_sarcasm = tokenizer.encode(test_sentence_not_sarcasm,
                                 truncation=True, padding=True, return_tensors="tf")

In [36]:
tf_output_not_sarcasm = loaded_model.predict(predict_input_not_sarcasm)[0]

In [37]:
tf_prediction_not_sarcasm = tf.nn.softmax(tf_output_not_sarcasm, axis=1).numpy()[0]

In [38]:
tf_prediction_not_sarcasm

array([0.9971927 , 0.00280738], dtype=float32)

In [52]:
tf_prediction = tf.nn.softmax(tf_output_not_sarcasm, axis=1).numpy()[0]

In [53]:
tf_prediction

array([0.9971927 , 0.00280738], dtype=float32)

Sarcasm Prediction

In [41]:
test_sentence_sarcasm = "News anchor hits back at viewer who sent her snarky note about ‘not performing well’ during broadcast"

In [46]:
predict_input_sarcasm = tokenizer.encode(test_sentence_sarcasm,
                                 truncation=True, padding=True, return_tensors="tf")

In [47]:
tf_output_sarcasm = loaded_model.predict(predict_input_sarcasm)[0]

In [48]:
tf_prediction_sarcasm = tf.nn.softmax(tf_output_sarcasm, axis=1).numpy()[0]

In [50]:
tf_prediction = tf.nn.softmax(tf_output_sarcasm, axis=1).numpy()[0]

In [51]:
tf_prediction

array([9.1376796e-04, 9.9908626e-01], dtype=float32)